In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd


### Company Data

In [2]:
database=pd.read_csv("stock_symbolRushi.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'stock_symbolRushi.csv'

In [ ]:
database

In [3]:
go ="https://www.moneycontrol.com/company-article/bharatforge/news/BF03"

In [4]:
# go=database["link"][3]
# go

In [5]:
res=requests.get(go)
res

<Response [200]>

In [6]:
soup=BeautifulSoup(res.text,"lxml")
l=soup.findAll("div",class_="pages MR10 MT15")
l

[<div align="right" class="pages MR10 MT15">
 <span class="current"><b>1</b></span> <a href="/stocks/company_info/stock_news.php?sc_id=BFC&amp;scat=&amp;pageno=2&amp;next=0&amp;durationType=M&amp;Year=&amp;duration=6&amp;news_type=">2</a> </div>]

In [7]:
newurl=[go]
def next_page_urls(l):
    global newurl
    for i in l:
        try:
            p=(i.a.get("href"))
        except AttributeError:
            return newurl
        else:
            main='https://www.moneycontrol.com'
            nexturl=main+p
            newurl.append(nexturl)
    return newurl

newurl= next_page_urls(l)

if newurl!=None:
    all_links=[]
    for page in newurl:
        nres=requests.get(page)
        nsoup=BeautifulSoup(nres.text,"lxml")
        nlinks=(nsoup.find_all("a",class_="g_14bl"))
        nlinks=list(nlinks)
        all_links.extend(nlinks)

print(len(all_links))

20


In [8]:
urls=[]
for i in range(len(all_links)):
    link=str(all_links[i])
    link=link[link.index("/news"):link.index('">\n<')]
    parent="https://www.moneycontrol.com"
    link=parent+link
    urls.append(link)

print(len(urls))

20


In [9]:
print(urls)

['https://www.moneycontrol.com/news/recommendations/hold-bharat-forge-targetrs-785-emkay-global-financial_16606581.html', 'https://www.moneycontrol.com/news/technicals/trade-spotlight-|-what-should-you-dobharat-forge-zomato-newgen-software-technologiestuesday_16585591.html', 'https://www.moneycontrol.com/news/technicals/trade-spotlight-|-what-should-you-dobharat-forge-zomato-newgen-software-technologiestuesday_16585581.html', 'https://www.moneycontrol.com/news/recommendations/buy-bharat-forge-targetrs-870-motilal-oswal_16584721.html', 'https://www.moneycontrol.com/news/recommendations/buy-bharat-forge-targetrs-900-icici-direct_16584591.html', 'https://www.moneycontrol.com/news/recommendations/buy-bharat-forge-targetrs-900-prabhudas-lilladher_16583381.html', 'https://www.moneycontrol.com/news/result-analysis/decent-setnumbers-by-bharat-forge_16581061.html', 'https://www.moneycontrol.com/news/results/bharat-forge-consolidated-june-2022-net-sales-at-rs-285146-crore3529-y-o-y_16581161.html

In [10]:
print(len(urls))
count=1
for i in urls:
    print(count)
    print(i)
    count+=1

20
1
https://www.moneycontrol.com/news/recommendations/hold-bharat-forge-targetrs-785-emkay-global-financial_16606581.html
2
https://www.moneycontrol.com/news/technicals/trade-spotlight-|-what-should-you-dobharat-forge-zomato-newgen-software-technologiestuesday_16585591.html
3
https://www.moneycontrol.com/news/technicals/trade-spotlight-|-what-should-you-dobharat-forge-zomato-newgen-software-technologiestuesday_16585581.html
4
https://www.moneycontrol.com/news/recommendations/buy-bharat-forge-targetrs-870-motilal-oswal_16584721.html
5
https://www.moneycontrol.com/news/recommendations/buy-bharat-forge-targetrs-900-icici-direct_16584591.html
6
https://www.moneycontrol.com/news/recommendations/buy-bharat-forge-targetrs-900-prabhudas-lilladher_16583381.html
7
https://www.moneycontrol.com/news/result-analysis/decent-setnumbers-by-bharat-forge_16581061.html
8
https://www.moneycontrol.com/news/results/bharat-forge-consolidated-june-2022-net-sales-at-rs-285146-crore3529-y-o-y_16581161.html
9
h

In [11]:
headlines=[]
subheadlines=[]
authors=[]
dates=[]
times=[]
articles=[]
for url in urls:
    response=requests.get(url).text
    news=BeautifulSoup(response,"lxml")

    #HEADING
    try:
        heading=news.find("h1",class_="article_title").text
    except AttributeError:
        pass
    else:
        headlines.append(heading)

    #SUBHEADING
    try:
        subheading=news.find("h2",class_="article_desc").text
    except AttributeError:
        pass
        
    else:
        if (len(subheading)>5):
            subheadlines.append(subheading)
        else:
            subheadlines.append(heading)

    #DATE
    try:
        date_time=news.find("div",class_="article_schedule").text

    except AttributeError:
        date=None
    else:
        date=date_time.split(" / ")[-2].strip()
        time=date_time.split(" / ")[-1].strip()
        times.append(time)
        dates.append(date)

    #AUTHOR

    
    try:
        author=news.find("div",class_="article_author").text
    except AttributeError:
        if date==None:
            pass
        else:
            author="No Author"
            authors.append("No Author")
    else:
        authors.append(author.strip())
        
    #ARTICLE
    try:
        news=news.find("div",class_="content_wrapper arti-flow")
        article=""
    except AttributeError:
        pass
    else:
        try:
            news.find('div',class_="social_icons_wrapper").decompose()
            news.find('div',class_="article_image_wrapper article_image_main_wrapper").decompose()
            news.find('div',class_="stockwidget").decompose()
            news.find('div',class_="hide-moblie mid-arti-ad").decompose()
            news.find('div',class_="show-moblie fixedfortopwrap").decompose()
        except AttributeError:
            pass
        else:
            try:
                news.find('div',class_="show-moblie").decompose()
            except AttributeError:
                pass
            try:
                news.find('div',class_="related_stories_left_block").decompose()
            except AttributeError:
                pass
        try:
            article = news.text
        except AttributeError:
            pass
        else:
            article=article.replace("For all recommendations report,\xa0click here","")
            article=article.replace("\r","")
            article=article[:article.find("Disclaimer")]
            # print(article.strip())
            articles.append(article.strip())
   

In [12]:
for count,text in enumerate(headlines,1):
    print(count)
    print(text)


1
REC seeks shareholders' nod to raise up to Rs 75,000 crore via bonds
2
REC gets shareholders' nod to issue bonus share
3
REC Standalone June 2022 Net Sales at Rs 9,459.90 crore, down 1.33% Y-o-Y
4
REC Consolidated June 2022 Net Sales at Rs 9,497.45 crore, down 1.39% Y-o-Y
5
REC Consolidated March 2022 Net Sales at Rs 9,633.37 crore, up 4.58% Y-o-Y
6
REC Standalone March 2022 Net Sales at Rs 9,580.10 crore, up 4.75% Y-o-Y


In [13]:
for count,text in enumerate(subheadlines,1):
    print(count)
    print(text)


1
The funds are proposed to be raised through issuance of debentures on private placement basis during a period of one year from the date of passing of the resolution, in one or more tranches, as per the AGM notice.
2
The company had proposed to issue one new (bonus) share for holding three existing shares to shareholders, as per a postal ballot notice of the REC.
3
REC Standalone June 2022 Net Sales at Rs 9,459.90 crore, down 1.33% Y-o-Y
4
REC Consolidated June 2022 Net Sales at Rs 9,497.45 crore, down 1.39% Y-o-Y
5
REC Consolidated March 2022 Net Sales at Rs 9,633.37 crore, up 4.58% Y-o-Y
6
REC Standalone March 2022 Net Sales at Rs 9,580.10 crore, up 4.75% Y-o-Y


In [14]:
for count,text in enumerate(dates,1):
    print(count)
    print(text)


1
September 05, 2022
2
August 10, 2022
3
August 05, 2022
4
August 05, 2022
5
May 16, 2022
6
May 16, 2022


In [15]:
for count,text in enumerate(times,1):
    print(count)
    print(text)


1
01:45 PM IST
2
02:21 PM IST
3
01:24 PM IST
4
01:09 PM IST
5
09:49 AM IST
6
09:07 AM IST


In [16]:
for count,text in enumerate(authors,1):
    print(count)
    print(text)


1
PTI
2
PTI
3
No Author
4
No Author
5
No Author
6
No Author


In [17]:
for count,text in enumerate(articles,1):
    print(count)
    print(text)


1
REC Ltd will seek shareholders' approval to raise up to Rs 75,000 crore through issuance of bonds next week. The funds are proposed to be raised through issuance of debentures on private placement basis during a period of one year from the date of passing of the resolution, in one or more tranches, as per the AGM notice.The limit of Rs 75,000 crore shall be within the overall revised borrowing limit. The annual general meeting (AGM) of shareholders is scheduled for September 16, 2022.REC will also seek shareholders' approval to authorize the board to mortgage/create charge on immovable and/or movable properties of the company, for securing loan up to Rs 4,50,000 crore.
2
State-owned REC has received shareholders' approval for the proposal to issue 65.83 crore bonus shares, utilising capital reserves of Rs 658.30 crore."...two resolutions as set out in the Postal Ballot Notice dated July 8, 2022, of the Company have been approved by the shareholders with requisite majority," a BSE fil

In [ ]:
data={"Date":dates,
     "Time":times,
     "Author":authors,
     "Headline":headlines,
     "Subheadlines":subheadlines,
     "Articles":articles}

df=pd.DataFrame(data)


In [ ]:
df